In [ ]:
#
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from   udist_scatterplots_via_sectors          import UDistScatterPlotsViaSectors
from   udist_scatterplots_via_sectors_tile_opt import UDistScatterPlotsViaSectorsTileOpt
import time
import copy
import rtsvg
rt = rtsvg.RACETrack()
import random

In [ ]:
_pts_ = [(1_000, 2_000, 500),
         (  100,   200,  50),
         (8_000,   100, 400)]
_its_ = [4, 16, 32]

#_pts_ = [(1_000, 2_000, 500)]
#_its_ = [4]

perfs = {}

for num_of_pts in _pts_:
    circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
    colors       = ['#ff0000','#006400','#0000ff']
    _xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
    for i in range(len(num_of_pts)):
        for j in range(num_of_pts[i]):
            a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
            x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
            _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])
    for _iterations_ in _its_:
        t0a = time.time()
        udspvs          = UDistScatterPlotsViaSectors       (_xvals_, _yvals_, iterations=_iterations_)
        t0b = time.time()

        k        = ('polars', num_of_pts, len(_xvals_), _iterations_, t0b-t0a)
        perfs[k] = copy.deepcopy(udspvs.time_lu)

        for _num_of_tiles_ in [16, 32, 64, 128, 256]:
            t1a = time.time()
            udspvs_tile_opt = UDistScatterPlotsViaSectorsTileOpt(_xvals_, _yvals_, iterations=_iterations_, num_of_tiles=_num_of_tiles_)
            t1b = time.time()

            k        = ('tile_opt', num_of_pts, len(_xvals_), _iterations_, t0b-t0a, _num_of_tiles_)
            perfs[k] = copy.deepcopy(udspvs_tile_opt.time_lu)

            print(f'{len(_xvals_):>5} Pts | {_iterations_:>3} Iters | {t0b-t0a:.2f}s Polars | {t1b-t1a:.2f}s Tile Opt ({_num_of_tiles_})')